In [50]:
import pandas  as pd
import sqlite3 as sql3

In [51]:
# Registrando colunas a serem selecionadas do arquivo csv
colunas = [i for i in range(0, 13)]

# Coletar e tratar tabela original do ibge
df = (
    pd
        # Coletando dados do arquivo csv
        .read_csv( 'uf_cidades_ibge.csv', sep = ';', usecols = colunas, dtype = str )
        
        # Renomeando colunas do DataFrame
        .rename(
            columns = {
                'UF'                                  : 'id_uf',
                'Nome_UF'                             : 'nome_uf',
                'Região Geográfica Intermediária'     : 'id_regiao_geografica_intermediaria',
                'Nome Região Geográfica Intermediária': 'nome_regiao_geografica_intermediaria',
                'Região Geográfica Imediata'          : 'id_regiao_geografica_imediata',
                'Nome Região Geográfica Imediata'     : 'nome_regiao_geografica_imediata',
                'Mesorregião Geográfica'              : 'id_mesorregiao_geografica',
                'Nome_Mesorregião'                    : 'nome_mesorregiao',
                'Microrregião Geográfica'             : 'id_microrregiao_geografica',
                'Nome_Microrregião'                   : 'nome_microrregiao_geografica',
                'Município'                           : 'id_municipio_curto',
                'Código Município Completo'           : 'id_municipio_completo',
                'Nome_Município'                      : 'nome_municipio'
            }
        )
)

In [52]:
# Coletando tabela com informações adicionais dos Estados, como Sigla e Região
info_ufs = (
    pd
        # Coletando tabela do site
        .read_html( 'https://www.estadosecapitaisdobrasil.com/' )[0]
        
        # Selecionando apenas colunas de interesse
        .iloc[ :, [1, 2, 4] ]
        
        # Renomeando colunas
        .rename(
            columns = {
                'Estado': 'nome_uf',
                'Sigla' : 'sigla_uf',
                'Região': 'regiao_uf'
            }
        )
)

In [53]:
# Unificando as duas tabelas
df = (
    df
        .merge(
            info_ufs,
            how = 'left',
            on  = 'nome_uf'
        )
)

In [54]:
# Reordenando as colunas do DataFrame
df = df.iloc[ :, [0, 1, 13, 14, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] ]

In [55]:
conn = sql3.connect( 'dados.bd' )

df.to_sql(
    'geografia.ibge',
    con       = conn,
    if_exists = 'replace',
    index     = False
)

5570